This is some boilerplate code for looking at the AnalyticsIQ data. If you need support, you can reach out to me on MS Teams, Haseeb Mohammed. 

You can find me on [LinkedIn](https://www.linkedin.com/in/bababrownbear/).
My email is haseeb@polaris.llc

Happy hacking!

Export tab 4 from the Excel file as CSV to start.

In [24]:
# Importing frequently used libraries when doing EDA (Exploratory Data Analysis)
import pandas as pd
import seaborn as sns
import numpy as np

In [20]:
pd.set_option('display.max_rows', 150) # See more than 5 rows by default when looking at data frames

data = pd.read_csv('AnalyticsIQ-SDoH-Data-Dictionary-Data-Sample-Hack4EquityGA.csv') # Tab 4 from the XLSX provided by AnalyticsIQ, save as CSV and upload here to Google Colab

In [21]:
data.head()

,zip5,zip4,county,city,Zip_Carrier_Route,Carrier_Route_Type_z4,Home_Age_z4,HomeValueIQ_z4,HomeValueIQ_Equity_z4,DwellingType_z4,Homeowner_Pct_z4,Affordability_Index_scf_z4,Affordability_Index_z4,Premover_v2_z4,Auto_Age_z4,Auto_Avg_Mileage_z4,Number_of_Autos_z4,InMarket_Aldi_z4,InMarket_BiLo_WinnDixie_z4,InMarket_Costco_z4,InMarket_H_E_B_z4,InMarket_Kroger_z4,InMarket_Meijer_z4,InMarket_Publix_z4,InMarket_Safeway_z4,InMarket_SamsClub_z4,InMarket_Sprouts_z4,InMarket_TraderJoes_z4,InMarket_Whole_Foods_z4,LT_Food_Delivery_Service_v2_z4,Grocery_Farmers_Mkt_z4,Grocery_Organic_z4,Grocery_Trips_z4,HW_BMI_z4,HW_Diet_z4,Diet_CageFree_Eggs_z4,Diet_Fake_Meat_Alt_z4,Diet_FreeRange_Chicken_z4,Diet_GrassFed_Beef_z4,Diet_Vegetarian_z4,...,OS_RiskTaking_Health_z4,HW_Alcohol_v2_z4,HW_Smoking_Cigars_SC_z4,HW_Smoking_Hookah_SC_z4,HW_Smoking_z4,HW_ER_Visits_SC_z4,HW_Urgent_Care_Visits_SC_z4,SCR_Wellness_z4,OS_Emotion_z4,InMarket_LTC_Ins_z4,Ins_ADaD_z4,Ins_Dental_z4,Ins_Homeowners_z4,Ins_Life_z4,Ins_Renters_z4,Ins_Supplemental_z4,Ins_Vision_z4,RegB_Ins_Auto_IM_z4,RegB_Ins_Health_IM_z4,RegB_Ins_Life_IM_z4,HW_Primary_Care_Doctor_SC_z4,RegB_Ins_Employer_AH_z4,RegB_Ins_Market_AH_z4,RegB_Ins_Medicare_AH_z4,Spendex_PERS_INS_v2_z4,HW_Med_Spec_Visits_SC_z4,HW_Med_Utilization_z4,HW_Sleep_v3_z4,HW_Stress_v2_z4,LT_Exercise_z4,HW_Telemedicine_SC_z4,Age_z4,Gender_Code_z4,EthnicIQ_v2_z4,Education_Coll_v2_z4,Education_Grad_v2_z4,Education_HS_v2_z4,Education_LTHS_v2_z4,NumberofAdultsinZip4,NumberHouseholdsInZip4
0,30002,2,Dekalb,Avondale Est,30002B001,B,67.0,NaN,NaN,Unknown,0,151.0,170.0,1.0,4.0,2.0,1.0,3.0,6.0,4.0,6.0,5.0,3.0,4.0,5.0,3.0,3.0,3.0,2.0,3.0,1.0,1.0,5.0,4.0,2.0,4.0,2.0,2.0,1.0,1.0,...,5.0,3.0,6.0,5.0,3.0,6.0,2.0,48.0,5.0,1.0,1.0,1.0,1.0,3.0,3.0,5.0,1.0,4.0,5.0,6.0,4.0,4.0,3.0,2.0,97.0,3.0,0.0,6.0,2.0,1.0,3.0,94.0,Male-Known,Caucasian-Other,0.0,0.0,100.0,0.0,1,1.0
1,30002,5,Dekalb,Avondale Est,30002B001,B,23.0,219.0,50.0,Single,0,151.0,170.0,5.0,4.0,6.0,2.0,2.0,6.0,5.0,6.0,4.0,3.0,4.0,4.0,5.0,2.0,4.0,4.0,4.0,4.0,3.0,3.0,4.0,4.0,4.0,6.0,5.0,5.0,4.0,...,6.0,4.0,4.0,4.0,2.0,5.0,4.0,53.0,7.0,4.0,5.0,5.0,1.0,7.0,4.0,4.0,5.0,6.0,6.0,7.0,5.0,4.0,5.0,2.0,405.0,6.0,0.0,7.0,4.0,1.0,4.0,55.0,Female-Inf,AA,0.0,0.0,100.0,0.0,3,1.0
2,30002,6,Dekalb,Avondale Est,30002B001,B,NaN,NaN,NaN,Unknown,0,151.0,170.0,4.0,4.0,4.0,1.0,7.0,6.0,5.0,4.0,4.0,4.0,4.0,5.0,5.0,4.0,6.0,5.0,4.0,6.0,3.0,5.0,4.0,4.0,4.0,4.0,3.0,2.0,2.0,...,7.0,6.0,6.0,6.0,3.0,5.0,3.0,44.0,4.0,4.0,5.0,5.0,1.0,4.0,6.0,2.0,4.0,7.0,7.0,7.0,4.0,3.0,5.0,2.0,340.0,3.0,0.0,3.0,4.0,2.0,4.0,56.0,Male-Known,Caucasian-Other,0.0,0.0,100.0,0.0,1,1.0
3,30002,9,Dekalb,Avondale Est,30002B001,B,NaN,310.0,310.0,Unknown,100,151.0,170.0,1.0,5.0,2.0,1.0,4.0,6.0,5.0,5.0,2.0,4.0,5.0,3.0,4.0,3.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,3.0,4.0,4.0,3.0,6.0,...,4.0,5.0,2.0,4.0,5.0,1.0,2.0,65.0,4.0,5.0,5.0,4.0,4.0,3.0,1.0,4.0,3.0,6.0,5.0,5.0,6.0,3.0,6.0,5.0,375.0,5.0,0.0,5.0,3.0,4.0,4.0,59.0,Female-Known,Caucasian-Other,0.0,0.0,100.0,0.0,1,1.0
4,30002,11,Dekalb,Avondale Est,30002B001,B,81.0,NaN,NaN,Single,0,151.0,170.0,2.0,5.0,3.0,2.0,2.0,6.0,5.0,6.0,3.0,4.0,4.0,3.0,4.0,3.0,4.0,3.0,4.0,4.0,3.0,4.0,4.0,5.0,5.0,5.0,6.0,4.0,6.0,...,5.0,3.0,3.0,2.0,2.0,5.0,4.0,51.0,5.0,2.0,4.0,3.0,1.0,3.0,5.0,6.0,2.0,5.0,6.0,7.0,7.0,4.0,5.0,5.0,1006.0,6.0,0.0,5.0,3.0,2.0,4.0,68.0,Female-Known,Caucasian-Other,0.0,0.0,50.0,50.0,2,1.0


In [22]:
data.shape # The total shape of the data. There are 259553 rows, and 128 columns.

(259553, 128)

In [23]:
def missing(dff):
    print (round((dff.isnull().sum() * 100/ len(dff)),2).sort_values(ascending=False))

missing(data) # Function for telling what portion of the data is missing per column

NumberOfChildren_z4               22.39
PersonsatResidence_z4             22.39
HomeValueIQ_z4                    21.88
HomeValueIQ_Equity_z4             21.88
Home_Age_z4                       20.81
InvestorIQ_Plus_v4_z4             13.26
WealthIQ_Plus_v4_z4                8.38
Age_z4                             1.28
AIQ_ATP_v2_z4                      0.31
NumberOfAdults_z4                  0.22
NumberHouseholdsInZip4             0.01
Affordability_Index_z4             0.01
OS_RiskTaking_Fin_z4               0.00
OS_Fin_Planning_z4                 0.00
OS_EPFI_z4                         0.00
HW_Job_Satis_z4                    0.00
Diet_FreeRange_Chicken_z4          0.00
HW_Junk_Diet_z4                    0.00
Diet_Vegetarian_z4                 0.00
Diet_GrassFed_Beef_z4              0.00
OS_Fin_IMP_z4                      0.00
Diet_Fake_Meat_Alt_z4              0.00
OS_Fin_IMPControl_z4               0.00
Invest_Real_Estate_z4              0.00
OS_Fin_Motivation_z4               0.00


In [27]:
df = df.dropna() # Drop NA columns

In [28]:
df.shape # Show new shape, 161005 rows

(161005, 128)

In [47]:
# Sample ML model, let's predict based on all the columns how likely someone will visit a medical specialist in the last 12 months.
predict = 'HW_Med_Spec_Visits_SC_z4' # Average Likelihood of Having Visited a Medical Specialist in Last 12-months (7=most likely; 1=least likely)
y = data[predict]
X = data.drop(columns=[predict])

In [49]:
y = y > 3.0 # The scale is from 1 to 7, assume 4 and higher as 'True'
y.value_counts() # The number of 'True' vs 'False'

True     141064
False     19941
Name: HW_Med_Spec_Visits_SC_z4, dtype: int64

In [50]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


for column in X.columns:
    if X[column].dtype == type(object):
        le = LabelEncoder()
        X[column] = le.fit_transform(X[column])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.80, random_state=101, stratify=y)

In [51]:
from sklearn import ensemble

params = {'n_estimators': 250, 'max_depth': 8, 'min_samples_split': 4,
          'learning_rate': 0.2, 'loss': 'ls'}
clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.2, loss='ls', max_depth=8,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=4,
                          min_weight_fraction_leaf=0.0, n_estimators=250,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [52]:
y_pred= clf.predict(X_test)
y_pred = y_pred > .4
y_pred = pd.DataFrame(y_pred)
y_pred = y_pred * 1
# y_pred.head()

In [53]:
from sklearn.metrics import (confusion_matrix, precision_score, recall_score, f1_score, classification_report)
# Metrics for classification

print(confusion_matrix(y_test, y_pred))

print(classification_report(y_test, y_pred))

print(recall_score(y_test, y_pred))

[[  9566   6387]
 [  2001 110850]]
              precision    recall  f1-score   support

       False       0.83      0.60      0.70     15953
        True       0.95      0.98      0.96    112851

    accuracy                           0.93    128804
   macro avg       0.89      0.79      0.83    128804
weighted avg       0.93      0.93      0.93    128804

0.9822686551293298
